In [24]:

#Importer les librairies nécessaires
import pandas as pd
import xml.etree.ElementTree as ET

In [25]:
# Chemin complet vers ton fichier XML sur le bureau
file_path = r'C:\Users\BOUQALLABA-TOUMMINI\Desktop\data_aqi_cpcb.xml'
df = pd.read_xml(r'C:\Users\BOUQALLABA-TOUMMINI\Desktop\data_aqi_cpcb.xml')
print(df)

      id     State
0  India  \n      


In [26]:
def xml_to_dataframe(file_path):

    # Charger et parser le fichier
    tree = ET.parse(file_path)  # adapte le nom du fichier
    root = tree.getroot()

    data = []
    # Explorer tous les éléments station

    for country in root.findall('.//Country'):
        for state in country.findall('.//State'):
            state_name = state.attrib.get('id', None)  
            for city in state.findall('.//City'):
                city_name = city.attrib.get('id', None)
                for station in city.findall('.//Station'):
                    station_name = station.attrib.get('id', None)
                    latitude = station.attrib.get('latitude', None)
                    longitude = station.attrib.get('longitude', None)
                    
                    station_data = {
                        'Country': country.attrib.get('id', None),  
                        'State': state_name,                       
                        'City': city_name,
                        'Station': station_name,
                        'Latitude': latitude,
                        'Longitude': longitude
                    }

                    # Polluants
                    for pollutant in station.findall('Pollutant_Index'):
                        pol_name = pollutant.attrib['id']
                        avg = pollutant.attrib['Avg']
                        station_data[f"{pol_name}_Avg"] = float(avg) if avg != "NA" else None

                    # AQI
                    aqi = station.find('Air_Quality_Index')
                    if aqi is not None:
                        station_data['AQI'] = int(aqi.attrib['Value'])
                        station_data['Predominant_Pollutant'] = aqi.attrib['Predominant_Parameter']
                        data.append(station_data)


    # Créer le DataFrame
    df = pd.DataFrame(data)
    return df


In [27]:
df = xml_to_dataframe(file_path)

#afficher le DataFrame
print(df.head())

  Country           State               City  \
0   India  Andhra_Pradesh          Anantapur   
1   India  Andhra_Pradesh           Chittoor   
2   India  Andhra_Pradesh             Kadapa   
3   India  Andhra_Pradesh  Rajamahendravaram   
4   India  Andhra_Pradesh           Tirupati   

                                          Station    Latitude   Longitude  \
0                    Gulzarpet, Anantapur - APPCB   14.675886   77.593027   
1             Gangineni Cheruvu, Chittoor - APPCB   13.204880   79.097889   
2                 Yerramukkapalli, Kadapa - APPCB   14.465052   78.824187   
3  Anand Kala Kshetram, Rajamahendravaram - APPCB  16.9872867  81.7363176   
4                      Tirumala, Tirupati - APPCB       13.67       79.35   

   PM2.5_Avg  PM10_Avg  NO2_Avg  NH3_Avg  SO2_Avg  CO_Avg  OZONE_Avg  AQI  \
0       24.0      24.0     23.0      3.0      7.0    29.0       25.0   29   
1       19.0      22.0     14.0      2.0      9.0     4.0       35.0   35   
2       37.0     

In [28]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Country                454 non-null    object 
 1   State                  454 non-null    object 
 2   City                   454 non-null    object 
 3   Station                454 non-null    object 
 4   Latitude               454 non-null    object 
 5   Longitude              454 non-null    object 
 6   PM2.5_Avg              448 non-null    float64
 7   PM10_Avg               442 non-null    float64
 8   NO2_Avg                436 non-null    float64
 9   NH3_Avg                388 non-null    float64
 10  SO2_Avg                425 non-null    float64
 11  CO_Avg                 437 non-null    float64
 12  OZONE_Avg              426 non-null    float64
 13  AQI                    454 non-null    int64  
 14  Predominant_Pollutant  454 non-null    object 
dtypes: flo

,PM2.5_Avg,PM10_Avg,NO2_Avg,NH3_Avg,SO2_Avg,CO_Avg,OZONE_Avg,AQI
count,448.000000,442.000000,436.000000,388.000000,425.000000,437.000000,426.000000,454.000000
mean,151.852679,132.361991,33.993119,6.974227,16.124706,47.048055,28.504695,161.555066
std,105.825806,76.433661,30.773215,7.368440,15.105009,27.342962,28.889912,99.428323
min,10.000000,16.000000,2.000000,1.000000,1.000000,4.000000,1.000000,15.000000
25%,63.000000,78.000000,14.000000,3.000000,7.000000,28.000000,12.000000,81.000000
50%,122.500000,117.000000,26.000000,5.000000,12.000000,41.000000,21.000000,137.000000
75%,232.500000,167.750000,44.000000,9.000000,19.000000,59.000000,35.000000,234.750000
max,416.000000,419.000000,245.000000,85.000000,126.000000,170.000000,220.000000,419.000000


In [29]:
#Les features (X) sont : PM2.5_Avg, PM10_Avg, NO2_Avg, NH3_Avg, SO2_Avg, CO_Avg, OZONE_Avg
# Le label (y) est : AQI (la valeur à prédire)
features_cols = ['PM2.5_Avg', 'PM10_Avg', 'NO2_Avg', 'NH3_Avg', 'SO2_Avg', 'CO_Avg', 'OZONE_Avg']
label_col = 'AQI'


In [30]:
for col in features_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')  # convertit 'NA' en NaN

# Garder que les lignes complètes 
df_clean = df.dropna(subset=features_cols + [label_col]) 



In [31]:
X = df_clean[features_cols]
y = df_clean[label_col]
print(df)


    Country           State               City  \
0     India  Andhra_Pradesh          Anantapur   
1     India  Andhra_Pradesh           Chittoor   
2     India  Andhra_Pradesh             Kadapa   
3     India  Andhra_Pradesh  Rajamahendravaram   
4     India  Andhra_Pradesh           Tirupati   
..      ...             ...                ...   
449   India     West_Bengal            Kolkata   
450   India     West_Bengal            Kolkata   
451   India     West_Bengal            Kolkata   
452   India     West_Bengal            Kolkata   
453   India     West_Bengal           Siliguri   

                                            Station    Latitude   Longitude  \
0                      Gulzarpet, Anantapur - APPCB   14.675886   77.593027   
1               Gangineni Cheruvu, Chittoor - APPCB   13.204880   79.097889   
2                   Yerramukkapalli, Kadapa - APPCB   14.465052   78.824187   
3    Anand Kala Kshetram, Rajamahendravaram - APPCB  16.9872867  81.7363176   
4   

## Entraîner un modèle de prédiction AQI

In [32]:
# Importer les outils de scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

### Explication :train_test_split: Permet de diviser les données en deux ensembles : un pour l'entraînement (train) et un pour les tests (test).LinearRegression: Le modèle de régression linéaire qui sera utilisé pour prédire les valeurs (dans ce cas, l'AQI).mean_squared_error: Calcul du MSE (Erreur quadratique moyenne) qui mesure la qualité de la prédiction du modèle.r2_score: Permet de calculer le score 𝑅2, qui mesure la proportion de la variance des données qui est expliquée par le modèle.

In [33]:
# Diviser les données en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
# Entraîner un modèle linéaire
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [35]:
y_pred = model.predict(X_test)

# Évaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R2 Score:", r2)

Mean Squared Error: 184.19327767428808
R2 Score: 0.9793300377336192


In [ ]:
print("Vraie valeur AQI:", y_test.values[0])
print("Valeur prédite AQI:", y_pred[0])


Vraie valeur AQI: 257
Valeur prédite AQI: 246.85035867164348


## Exemple de prédiction avec de nouvelles valeurs

In [37]:
new_data = pd.DataFrame({
    'PM2.5_Avg': [40],
    'PM10_Avg': [50],
    'NO2_Avg': [25],
    'NH3_Avg': [5],
    'SO2_Avg': [10],
    'CO_Avg': [20],
    'OZONE_Avg': [30]
})

# Faire une prédiction pour ces nouvelles données
predicted_aqi = model.predict(new_data)

# Afficher la prédiction de l'AQI
print(f"Prédiction de l'AQI pour les nouvelles données : {predicted_aqi[0]}")

Prédiction de l'AQI pour les nouvelles données : 55.27310436813182
